In [ ]:
#!pip install opencv-python
#!pip install -q tf-focal-frequency-loss
import tensorflow as tf
import numpy as np
import keras
import pathlib
from tensorflow.keras import layers
from matplotlib import pyplot as plt
from IPython import display
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tf_focal_frequency_loss import FocalFrequencyLoss as FFL

#!rm -r ~/.kaggle
#!mkdir ~/.kaggle
#!mv ./kaggle.json ~/.kaggle/
#!chmod 600 ~/.kaggle/kaggle.json
#!kaggle datasets download -d kmader/skin-cancer-mnist-ham10000

#!unzip /content/skin-cancer-mnist-ham10000.zip

import cv2
import matplotlib.pyplot as plt

def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(256, input_shape=(100,), use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Dense(512, use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Dense(12288, activation='tanh'))

    return model


def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Flatten())
    model.add(layers.Dense(512))
    model.add(layers.LeakyReLU())
    model.add(layers.Dense(256))
    model.add(layers.LeakyReLU())
    model.add(layers.Dense(1, activation='sigmoid'))

    return model

cross_entropy = tf.keras.losses.BinaryCrossentropy()

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

@tf.function
def train_step(images):
    noise = tf.random.normal([batch_size, 100])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

def generate_images(model, test_input):
    predictions = model(test_input, training=False)
    fig = plt.figure(figsize=(4, 4))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow((predictions[i, :, :] + 1) / 2.0)
        plt.axis('off')

    plt.show()

image_size = (64, 64)
batch_size = 256
data_generator = ImageDataGenerator(rescale=1.0 / 255)
train_dataset = data_generator.flow_from_directory(
    '/content/Images',
    target_size=image_size,
    batch_size=batch_size,
    class_mode=None,
    shuffle=True
)



num_images = 100
selected_images = []
for i in range(num_images):
    selected_images.append(next(train_dataset))
selected_images = np.concatenate(selected_images, axis=0)
selected_images = selected_images[:num_images]

generator = make_generator_model()
discriminator = make_discriminator_model()

def train(dataset, epochs):
    for epoch in range(epochs):
        for image_batch in dataset:
            train_step(image_batch)

        if (epoch + 1) % 10 == 0:
            generate_images(generator, tf.random.normal([16, 100]))

EPOCHS = 100
train(train_dataset, EPOCHS)




Found 10015 images belonging to 2 classes.
